#### Importacion de librerias: 

In [607]:
import sim 
import simConst 
import numpy as np 
import sympy as sp  

#### Establecemos la conexion: 

In [608]:
def connect(port): 
    sim.simxFinish(-1) #Desconecta todos los puertos. 
    clientID = sim.simxStart('127.0.0.1', port, True, True, 2000, 5)
    if clientID == 0: 
        print("Conectado a : ", port)
    else: 
        print("No se pudo conectar a: ", port)
    return clientID 

In [609]:
clientID = connect(1111)

Conectado a :  1111


Handle Joint

In [635]:
ret, joint1_handle = sim.simxGetObjectHandle(clientID, "joint1", sim.simx_opmode_blocking)
ret, joint2_handle = sim.simxGetObjectHandle(clientID, "joint2", sim.simx_opmode_blocking)
ret, joint3_handle = sim.simxGetObjectHandle(clientID, "joint3", sim.simx_opmode_blocking)

ret, base = sim.simxGetObjectHandle(clientID, "link1_visible", sim.simx_opmode_blocking)
ret, pos_base = sim.simxGetJointPosition(clientID, base, sim.simx_opmode_blocking)
print("pos base: ", pos_base)

ret, pos_joint0 = sim.simxGetJointPosition(clientID, joint1_handle, sim.simx_opmode_blocking)
ret, pos_joint1 = sim.simxGetJointPosition(clientID, joint2_handle, sim.simx_opmode_blocking)
ret, pos_joint3 = sim.simxGetJointPosition(clientID, joint3_handle, sim.simx_opmode_blocking)
print(f"Posicion Joint1: {pos_joint0}, Posicion Joint2: {pos_joint1}")


Posicion Joint1: 2.967059850692749, Posicion Joint2: 1.5717215538024902


### Cinematica directa 

In [639]:
## Longitud de los eslabones: 
d1 = 0.1
d2 = 0.1
d3 = 0.1

q1 = np.radians(0)
q2 = np.radians(0)
q3 = np.radians(0)
ret = sim.simxSetJointTargetPosition(clientID, joint1_handle, q1, sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint2_handle, q2, sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint3_handle, q3, sim.simx_opmode_blocking)
#Cinematica directa:
# Si conozco las posiciones de los angulos puedo conocer
# la pos final del robot.  
# x = d3*np.sin(q0)*np.sin(q1)
# y = -d3*np.cos(q0)*np.sin(q1)
# z = d1+d2+d3*np.cos(q1)
# print(f"Pos con cinematica en x: {x}, pos en y: {y}, pos en z: {z}") 
retrunCode, pos_final_handle = sim.simxGetObjectHandle(clientID, "pos_final", sim.simx_opmode_blocking)
pos_final = sim.simxGetObjectPosition(clientID, pos_final_handle, -1, sim.simx_opmode_blocking)
print("Posicion del extremo final: ", pos_final)

Posicion del extremo final:  (0, [-0.014157406985759735, 6.27084809821099e-05, 0.3150109052658081])


#### Cinematica inversa FABRIK

In [612]:
def pendiente(segments):
    print("Encontrar los angulos entre los sig puntos: ", segments)
    eje_y = []
    eje_x = [] 
    q = []
    for i in range(len(segments) - 1): 
        eje_y.append(segments[i+1]["y"] - segments[i]["y"])
        eje_x.append(segments[i+1]["x"] - segments[i]["x"])

    for y, x in zip(eje_y, eje_x): 
        if y/x < 0.0:
            print("La pendiente es negativa") 
            q.append(np.arctan2(y,x))
        else: 
            print("La pendiente es positiva") 
            q.append(np.arctan2(y,x))
        print("q: ", q)
    return q  

In [613]:
def producto_punto(segments):
    #[p2, p1, p0]
    segments.reverse()
    eje_y = []
    eje_x = []
    producto_punto = [] 
    q = []
    for i in range(len(segments) - 1): 
        eje_y.append(segments[i]["y"] - segments[i + 1]["y"])
        eje_x.append(segments[i]["x"] - segments[i + 1]["x"])

    #Producto punto: 
    for y, x in zip(eje_y, eje_x): 
        producto_punto.append(x*x)
        print("y", y)
        print("x", x)
        print("Producto punto: ", producto_punto)
    
    for producto, x, y in zip(producto_punto, eje_x, eje_y): 
        resultado = np.arccos(
            producto/(np.sqrt(x**2 + y**2) * x)
        )
        q.append(resultado)
    print("q: ", q)
    return q

In [614]:
#Fabrik
def fabrik(dist_vectores, segments, tgt, base):     
    base = base 
    segments = segments 
    dist_vectores = dist_vectores
    p_prima = [tgt]
    
    for i in range(len(segments) - 1): 
        eje_x = segments[i+1]["x"] - p_prima[i]["x"] 
        eje_y = segments[i+1]["y"] - p_prima[i]["y"]
        ux = eje_x/(np.sqrt(eje_x**2 + eje_y**2))
        uy = eje_y/(np.sqrt(eje_x**2 + eje_y**2))

        prima_x = p_prima[i]["x"] + dist_vectores[i]*ux
        prima_y = p_prima[i]["y"] + dist_vectores[i]*uy
        prima = {
            "x": prima_x, 
            "y": prima_y
        }
        p_prima.append(prima)

    #Backward:
    # [d1 ,d2]
    dist_vectores.reverse()
    print("p_prima:", len(p_prima))
    p_prima.reverse()
    p_2prima = [base] 
    print("Distancia de vectores: ", dist_vectores)

    for i in range(len(segments) - 1): 
        eje_x = p_prima[i+1]["x"] - p_2prima[i]["x"] 
        eje_y = p_prima[i+1]["y"] - p_2prima[i]["y"]
        ux = eje_x/(np.sqrt(eje_x**2 + eje_y**2))
        uy = eje_y/(np.sqrt(eje_x**2 + eje_y**2))

        prima_x = p_2prima[i]["x"] + dist_vectores[i]*ux
        prima_y = p_2prima[i]["y"] + dist_vectores[i]*uy
        prima = {
            "x": prima_x, 
            "y": prima_y
        }
        p_2prima.append(prima)

    
    print("p_2prima:", p_2prima)
    
    return p_2prima   

In [615]:
# Condiciones iniciales: 
p0 = {
  "x": 0, 
  "y": 0
}
p1 = {
  "x": 1, 
  "y": 0
}
p2 = {
  "x": 1.5, 
  "y": 0
}

In [616]:
#main Fabrik
retrunCode, target_handle = sim.simxGetObjectHandle(clientID, "target", sim.simx_opmode_blocking)
returnCode, pos_target = sim.simxGetObjectPosition(clientID, target_handle, -1, sim.simx_opmode_blocking)
d1 = 1
d2 = 0.5
tgt = {
  "x": pos_target[0], 
  "y": pos_target[1], 
}

dist_vectores = [d2, d1]
segments = [p2, p1, p0] 
base = segments[len(segments) - 1]

for i in range(2): 
  p_2prima = fabrik(dist_vectores, segments, tgt, base)
  segments = p_2prima 
  dist_vectores = [d2, d1]
  print("Segments news: ", segments)
  

p_prima: 3
Distancia de vectores:  [1, 0.5]
p_2prima: [{'x': 0, 'y': 0}, {'x': 0.9900022840978616, 'y': 0.14105132924228989}, {'x': 0.49002378071136815, 'y': 0.14568772346648406}]
Segments news:  [{'x': 0, 'y': 0}, {'x': 0.9900022840978616, 'y': 0.14105132924228989}, {'x': 0.49002378071136815, 'y': 0.14568772346648406}]
p_prima: 3
Distancia de vectores:  [1, 0.5]
p_2prima: [{'x': 0, 'y': 0}, {'x': 0.9637371397841633, 'y': 0.2668533780948636}, {'x': 0.46756647271907154, 'y': 0.20509037020031654}]
Segments news:  [{'x': 0, 'y': 0}, {'x': 0.9637371397841633, 'y': 0.2668533780948636}, {'x': 0.46756647271907154, 'y': 0.20509037020031654}]


In [617]:
#Calculo de los angulos entro los puntos dados por FABRIK: 
def angulos_fabrik(p_2prima): 
    y = float(p_2prima[1]["y"])
    x = float(p_2prima[1]["x"])

    q = np.arcsin(y)
    print("Valor de q1: ", q)
    eje_x = d1*np.cos(q)
    eje_y = d1*np.sin(q)
    print("cinematica directa eslabon 1 eje_x: ", eje_x)
    print(f"cinematica directa eslabon 1: eje_y {eje_y} \n")
    if x < 0:  
        print("El punto x es negativo: ")
        eje_x = d1*np.cos(np.pi - q)
        eje_y = d1*np.sin(np.pi - q)
        q = np.pi - q
        print("cinematica directa eslabon 1 eje_x: ", eje_x)
        print(f"cinematica directa eslabon 1: eje_y {eje_y} \n")

# Eslabon 2
    y = float(p_2prima[2]["y"])
    x = float(p_2prima[2]["x"])
    q2 = np.arcsin((y - d1*np.sin(q))/d2) - q

    print("Valor de q2: ", q2)
    eje_x2 = d2*np.cos(q + q2) + d1*np.cos(q)
    eje_y2 = d2*np.sin(q + q2) + d1*np.sin(q)
    print("cinematica directa eslabon 2 eje_x: ", eje_x2)
    print(f"cinematica directa eslabon 2: eje_y {eje_y2}")
    if abs(x - eje_x2) > 0.2: 
        print("El eje x del esl2 es distinto, probando con recorrido for")
        for q2 in np.arange(0, np.pi, 0.0001): 
            if (abs(d2*np.cos(q + q2) + d1*np.cos(q) - x) < 0.001) and (abs(d2*np.sin(q + q2) + d1*np.sin(q) - y) < 0.001): 
                break
            else: 
                print("No se encontro un angulo que satisface la situación")
        print(q2)
        eje_x2 = d2*np.cos(q + q2) + d1*np.cos(q)
        eje_y2 = d2*np.sin(q + q2) + d1*np.sin(q)
        print("cinematica directa eslabon 2 eje_x: ", eje_x2)
        print(f"cinematica directa eslabon 2: eje_y {eje_y2}")
    print(f"Segments: {p_2prima} \n") 

    return [q, q2]

In [618]:
#Calculo de los angulos entre los eslabones: 
#q = producto_punto(p_2prima)
#q = pendiente(p_2prima)
q = angulos_fabrik(p_2prima)

#Movimiento de los motores:  
ret = sim.simxSetJointTargetPosition(clientID, joint1_handle, q[0], sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint2_handle, q[1], sim.simx_opmode_blocking)

Valor de q1:  0.27012652752007543
cinematica directa eslabon 1 eje_x:  0.9637371397841632
cinematica directa eslabon 1: eje_y 0.2668533780948636 

Valor de q2:  -0.3939688607097434
cinematica directa eslabon 2 eje_x:  1.4599078068492548
cinematica directa eslabon 2: eje_y 0.20509037020031654
El eje x del esl2 es distinto, probando con recorrido for
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo que satisface la situación
No se encontro un angulo 

In [619]:
retrunCode, pos_final_handle = sim.simxGetObjectHandle(clientID, "pos_final", sim.simx_opmode_blocking)
retCode, pos_final = sim.simxGetObjectPosition(clientID, pos_final_handle, -1, sim.simx_opmode_blocking)
etrunCode, eslabon1_handle = sim.simxGetObjectHandle(clientID, "eslabon1", sim.simx_opmode_blocking)
retCode, pos_eslabon1 = sim.simxGetObjectPosition(clientID, eslabon1_handle, -1, sim.simx_opmode_blocking)
print("Posicion del extremo final: ", pos_final)
print("Target: [x,y,z] ", tgt)
print("Eslabon1: ", pos_eslabon1)
print("Segmentos encontrados por Fabrik: ")
print("Ultimo segmento: ", p_2prima[0])
print("Segments: ", p_2prima)


Posicion del extremo final:  [-0.21758154034614563, 0.03813973814249039, 0.02765670046210289]
Target: [x,y,z]  {'x': 0.024999983608722687, 'y': 0.1499999761581421}
Eslabon1:  [-0.6043200492858887, 0.547685444355011, 0.447897732257843]
Segmentos encontrados por Fabrik: 
Ultimo segmento:  {'x': 0, 'y': 0}
Segments:  [{'x': 0, 'y': 0}, {'x': 0.9637371397841633, 'y': 0.2668533780948636}, {'x': 0.46756647271907154, 'y': 0.20509037020031654}]
